In [23]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
import tracker
latest_date = tracker.processing_dates[-1]
os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [24]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/fastq/{}.fastq.samplesheet.with_header.tsv'.format(latest_date)
    output_prefix = 'results/samplesheets/fastq/{}.fastq.post-check.download_srr_fastqs'.format(latest_date)
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

## Helper Functions

In [25]:
def list_to_str(l, sep=' '):
    l = [str(x) for x in l]
    return(sep.join(l))

def read_log(log):
    
    """
    Read the log and extract status information including
    the sample_name, srr_id, status (as indicated by the end message),
    and the download status (as indicate by the "reads written" message).
    """
    
    status = 0
    download = 1
    sample_name = 'check'
    srr_id = 'check'
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            elif 'srr_id' in info:
                srr_id = info.split()[1]
            elif 'Ended: download_srr_fastqs' in info:
                status = 1
            elif 'reads written' in info:
                download = 1
            
    return([index, sample_name, srr_id, job_id, status, download])

def long_to_short_index_list(serial_list):
    
    small_num = serial_list[0]
    serial_list_short = []
    for i in range(1, len(serial_list)):

        prev_num = serial_list[i -1]
        next_num = serial_list[i]

        if (prev_num + 1) != next_num:

            if small_num == prev_num:
                curr_range = '{}'.format(small_num)
                serial_list_short.append(curr_range)
                small_num = next_num

            else:
                curr_range = '{}-{}'.format(small_num, prev_num)
                serial_list_short.append(curr_range)
                small_num = next_num

    if small_num == next_num:
        curr_range = '{}'.format(small_num,)
        serial_list_short.append(curr_range)
    else:
        curr_range = '{}-{}'.format(small_num, next_num)
        serial_list_short.append(curr_range)
        
    return(serial_list_short)

## Check Logs

In [26]:
# load the data
df = pd.read_table(input_fn)
df['sample_index'] = df.index + 1 # add 1 since we'll be using 1-based indexing with sed

In [27]:
df

,std_sample_name,gse_id,gsm_id,srr_id,organism,bio_rep,tech_rep,antibody_target,restriction_enzyme,sample_name,sample_index
0,HAVIC.GSE154512.Homo_Sapiens.H3K27ac.b1,GSE154512,GSM4672364,SRR12231664,Homo_Sapiens,1,1,H3K27ac,MboI,HAVIC,1
1,HCT116-AuxinNeg.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,GSE179544,GSM5420289,SRR15050696,Homo_Sapiens,1,1,RNA-Pol-II,DpnII,HCT116-AuxinNeg,2
2,HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,GSE179544,GSM5420290,SRR15050697,Homo_Sapiens,1,1,RNA-Pol-II,DpnII,HCT116-AuxinPos,3
3,HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,GSE179544,GSM5420292,SRR15050699,Homo_Sapiens,1,2,RNA-Pol-II,DpnII,HCT116-AuxinPos,4
4,IMR90-Proliferating.GSE100856.Homo_Sapiens.CTCF.b1,GSE100856,GSM2695301,SRR5808478,Homo_Sapiens,1,1,CTCF,HindIII,IMR90-Proliferating,5
5,IMR90-Senescent.GSE100856.Homo_Sapiens.CTCF.b1,GSE100856,GSM2695302,SRR5808479,Homo_Sapiens,1,1,CTCF,HindIII,IMR90-Senescent,6
6,HUVEC-Proliferating.GSE100856.Homo_Sapiens.CTCF.b1,GSE100856,GSM2936365,SRR6496514,Homo_Sapiens,1,1,CTCF,HindIII,HUVEC-Proliferating,7
7,HUVEC-Senescent.GSE100856.Homo_Sapiens.CTCF.b1,GSE100856,GSM2936366,SRR6496515,Homo_Sapiens,1,1,CTCF,HindIII,HUVEC-Senescent,8
8,Hela-S3-Control.GSE108869.Homo_Sapiens.CTCF.b1,GSE108869,GSM2974085,SRR6657726,Homo_Sapiens,1,1,CTCF,MboI,Hela-S3-Control,9
9,Hela-S3-Control.GSE108869.Homo_Sapiens.CTCF.b2,GSE108869,GSM2974086,SRR6657727,Homo_Sapiens,2,1,CTCF,MboI,Hela-S3-Control,10


In [28]:
# create a dataframe of log information 
logs = glob.glob('results/fastqs/raw/logs/download_srr_fastqs.o*-*')
log_data = []
for log in logs:
    index, sample_name, srr_id, job_id, status, download = read_log(log)
    log_data.append([index, sample_name, srr_id, job_id, status, download, log])
log_df = pd.DataFrame(log_data)
log_df.columns = ['sample_index', 'std_sample_name', 'srr_id', 'job_id', 'end_status', 'read_written_status', 'log']
log_df.drop('sample_index', inplace=True, axis=1)

In [29]:
# merge the log information
df = df.merge(log_df, how='left', on=['std_sample_name', 'srr_id'], indicator='merge')
df.loc[df.end_status.isna(), 'end_status'] = 0
df.loc[df.read_written_status.isna(), 'read_written_status'] = 0
df.end_status = df.end_status.astype(int)
df.read_written_status = df.read_written_status.astype(int)

In [30]:
# get the latest job id for a give sample + srr id combo
df = df.sort_values('job_id', ascending=False).drop_duplicates(['std_sample_name', 'srr_id'])

# sort back into the sample index values, needed for correctly running qsub
df.sort_values('sample_index', inplace=True)

## Check the presences of SRR FASTQ files

In [31]:
# get the download path
download_tpl = 'results/fastqs/raw/{}/{}_2.fastq.gz'
df.loc[:, 'download_output'] = df.apply(lambda x: download_tpl.format(x[0], x[3]), axis=1)

# find whether the download SRR FASTQ is present
df.loc[:, 'download_present'] = df.loc[:, 'download_output'].apply(os.path.exists).astype(int)

In [32]:
reorder_cols = ['std_sample_name',
                 'gse_id',
                 'gsm_id',
                 'srr_id',
                 'organism',
                 'bio_rep',
                 'tech_rep',
                 'sample_index',
                 'job_id',
                 'end_status',
                 'read_written_status',
                 'download_present',
                 'antibody_target',
                 'restriction_enzyme',
                 'sample_name',
                 'log',
                 'download_output']

In [33]:
df = df[reorder_cols]

In [34]:
save_df = df.rename(columns={'end_status': 'download_end_status', 
                   'read_written_status': 'download_read_written_status',
                   'log': 'download_log', 
                   'job_id': 'download_job_id'})

header_output = '{}.with_header.tsv'.format(output_prefix)
save_df.to_csv(header_output, header=True, index=False, sep='\t')

without_header_output = '{}.without_header.tsv'.format(output_prefix)
save_df.to_csv(without_header_output, header=False, index=False, sep='\t')

## Identifying problem samples

In [44]:
problems = (df.end_status != 1) | (df.read_written_status != 1) | (df.download_present != 1)
problems_df = df.loc[problems,:]
problems_df

,std_sample_name,gse_id,gsm_id,srr_id,organism,bio_rep,tech_rep,sample_index,job_id,end_status,read_written_status,download_present,antibody_target,restriction_enzyme,sample_name,log,download_output
74,Nalm6.GSE115492.Homo_Sapiens.H3K27ac.b1,GSE115492,GSM3179300,SRR7278684,Homo_Sapiens,1,1,20,5028246,1,1,0,H3K27ac,MboI,Nalm6,results/fastqs/raw/logs/download_srr_fastqs.o5028246-20,results/fastqs/raw/Nalm6.GSE115492.Homo_Sapiens.H3K27ac.b1/SRR7278684_2.fastq.gz
326,PAEC-siRNA-Ctrl.GSE152900.Homo_Sapiens.H3K27ac.b1,GSE152900,GSM5988131,SRR18552251,Homo_Sapiens,1,1,177,5028246,1,1,0,H3K27ac,MboI,PAEC-siRNA-Ctrl,results/fastqs/raw/logs/download_srr_fastqs.o5028246-177,results/fastqs/raw/PAEC-siRNA-Ctrl.GSE152900.Homo_Sapiens.H3K27ac.b1/SRR18552251_2.fastq.gz
328,PAEC-siRNA-Ctrl.GSE152900.Homo_Sapiens.H3K27ac.b2,GSE152900,GSM5988132,SRR18552250,Homo_Sapiens,2,1,178,5028246,1,1,0,H3K27ac,MboI,PAEC-siRNA-Ctrl,results/fastqs/raw/logs/download_srr_fastqs.o5028246-178,results/fastqs/raw/PAEC-siRNA-Ctrl.GSE152900.Homo_Sapiens.H3K27ac.b2/SRR18552250_2.fastq.gz
353,Primary-Foreskin-Keratinocytes-Day3.GSE158642.Homo_Sapiens.H3K27ac.b1,GSE158642,GSM4804909,SRR12728609,Homo_Sapiens,1,1,192,5028246,1,1,0,H3K27ac,MboI,Primary-Foreskin-Keratinocytes-Day3,results/fastqs/raw/logs/download_srr_fastqs.o5028246-192,results/fastqs/raw/Primary-Foreskin-Keratinocytes-Day3.GSE158642.Homo_Sapiens.H3K27ac.b1/SRR12728609_2.fastq.gz
408,hESC-AHDC1-GIBBIN-KO.GSE181971.Homo_Sapiens.SMC1A.b3,GSE181971,GSM5515078,SRR15425305,Homo_Sapiens,3,1,226,5028246,1,1,0,SMC1A,MboI,hESC-AHDC1-GIBBIN-KO,results/fastqs/raw/logs/download_srr_fastqs.o5028246-226,results/fastqs/raw/hESC-AHDC1-GIBBIN-KO.GSE181971.Homo_Sapiens.SMC1A.b3/SRR15425305_2.fastq.gz
423,THP1-CTRL.GSE188380.Homo_Sapiens.CTCF.b3,GSE188380,GSM5678430,SRR16849265,Homo_Sapiens,3,1,237,5028246,0,1,0,CTCF,MboI,THP1-CTRL,results/fastqs/raw/logs/download_srr_fastqs.o5028246-237,results/fastqs/raw/THP1-CTRL.GSE188380.Homo_Sapiens.CTCF.b3/SRR16849265_2.fastq.gz
429,THP1-125D.GSE188380.Homo_Sapiens.CTCF.b1,GSE188380,GSM5678431,SRR16849266,Homo_Sapiens,1,1,238,5028246,0,1,0,CTCF,MboI,THP1-125D,results/fastqs/raw/logs/download_srr_fastqs.o5028246-238,results/fastqs/raw/THP1-125D.GSE188380.Homo_Sapiens.CTCF.b1/SRR16849266_2.fastq.gz
432,THP1-125D.GSE188380.Homo_Sapiens.CTCF.b2,GSE188380,GSM5678432,SRR16849267,Homo_Sapiens,2,1,239,5028246,0,1,0,CTCF,MboI,THP1-125D,results/fastqs/raw/logs/download_srr_fastqs.o5028246-239,results/fastqs/raw/THP1-125D.GSE188380.Homo_Sapiens.CTCF.b2/SRR16849267_2.fastq.gz
446,GM12878.GSE80820.Homo_Sapiens.SMC1A.b2,GSE80820,GSM2138326,SRR3467177,Homo_Sapiens,2,1,245,5028246,1,1,0,SMC1A,MboI,GM12878,results/fastqs/raw/logs/download_srr_fastqs.o5028246-245,results/fastqs/raw/GM12878.GSE80820.Homo_Sapiens.SMC1A.b2/SRR3467177_2.fastq.gz


In [45]:
problems_df.loc[:, ['std_sample_name', 'log']]

,std_sample_name,log
74,Nalm6.GSE115492.Homo_Sapiens.H3K27ac.b1,results/fastqs/raw/logs/download_srr_fastqs.o5028246-20
326,PAEC-siRNA-Ctrl.GSE152900.Homo_Sapiens.H3K27ac.b1,results/fastqs/raw/logs/download_srr_fastqs.o5028246-177
328,PAEC-siRNA-Ctrl.GSE152900.Homo_Sapiens.H3K27ac.b2,results/fastqs/raw/logs/download_srr_fastqs.o5028246-178
353,Primary-Foreskin-Keratinocytes-Day3.GSE158642.Homo_Sapiens.H3K27ac.b1,results/fastqs/raw/logs/download_srr_fastqs.o5028246-192
408,hESC-AHDC1-GIBBIN-KO.GSE181971.Homo_Sapiens.SMC1A.b3,results/fastqs/raw/logs/download_srr_fastqs.o5028246-226
423,THP1-CTRL.GSE188380.Homo_Sapiens.CTCF.b3,results/fastqs/raw/logs/download_srr_fastqs.o5028246-237
429,THP1-125D.GSE188380.Homo_Sapiens.CTCF.b1,results/fastqs/raw/logs/download_srr_fastqs.o5028246-238
432,THP1-125D.GSE188380.Homo_Sapiens.CTCF.b2,results/fastqs/raw/logs/download_srr_fastqs.o5028246-239
446,GM12878.GSE80820.Homo_Sapiens.SMC1A.b2,results/fastqs/raw/logs/download_srr_fastqs.o5028246-245


In [46]:
problems_df.shape

(9, 17)

In [47]:
df.shape

(262, 17)

## Rerun problematic samples

In [48]:
# convert not run serial IDs to ranges/short format
rerun_indexes = sorted(problems_df.sample_index.tolist())
rerun_indexes = long_to_short_index_list(rerun_indexes)

In [49]:
len(rerun_indexes)

6

In [40]:
# generate a string version of not_run
final_rerun = ','.join(rerun_indexes)
final_rerun = 'qsub -t {}%4 workflow/scripts/fastq/download_srr_fastqs.qarray.qsh'.format(final_rerun)

In [41]:
print(final_rerun)

qsub -t 20,177-178,192,226,237-239,245%4 workflow/scripts/fastq/download_srr_fastqs.qarray.qsh


<p style="color: red"><b>Re-running in progress: 2022.04.02 - 16:30</b></p>

In [42]:
'vim -p {}'.format(' '.join(problems_df.log.tolist()))

'vim -p results/fastqs/raw/logs/download_srr_fastqs.o5028246-20 results/fastqs/raw/logs/download_srr_fastqs.o5028246-177 results/fastqs/raw/logs/download_srr_fastqs.o5028246-178 results/fastqs/raw/logs/download_srr_fastqs.o5028246-192 results/fastqs/raw/logs/download_srr_fastqs.o5028246-226 results/fastqs/raw/logs/download_srr_fastqs.o5028246-237 results/fastqs/raw/logs/download_srr_fastqs.o5028246-238 results/fastqs/raw/logs/download_srr_fastqs.o5028246-239 results/fastqs/raw/logs/download_srr_fastqs.o5028246-245'